# Use IMPA for unseen drug prediction BBBC021 

Import libraries 

In [1]:
# Standard library imports
import os
from pathlib import Path

# Third-party library imports
from tutorial_utils import t2np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yaml 
import torch
from omegaconf import OmegaConf
from tqdm import tqdm

# Local application/library imports
from IMPA.dataset.data_loader import CellDataLoader
from IMPA.solver import IMPAmodule

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


Read the configuration of interest

In [2]:
path_to_config = "../config_hydra/config/bbbc021_all.yaml"

In [3]:
# Reading the YAML file
with open(path_to_config, 'r') as file:
    config = yaml.safe_load(file)  # Use safe_load to avoid executing arbitrary code

# Access the loaded data
print(config)

{'task_name': 'bbbc021_all', 'img_size': 128, 'latent_dim': 1024, 'hidden_dim': 512, 'style_dim': 64, 'stochastic': True, 'z_dimension': 8, 'dim_in': 64, 'lambda_reg': 1, 'lambda_cyc': 1, 'lambda_sty': 1, 'lambda_ds': 1, 'total_epochs': 200, 'ds_iter': 100000, 'resume_iter': 0, 'batch_size': 16, 'val_batch_size': 8, 'lr': 0.0001, 'f_lr': 0.0001, 'beta1': 0, 'beta2': 0.99, 'weight_decay': 0.0001, 'num_outs_per_domain': 10, 'single_style': True, 'ood_set': ['taxol', 'ALLN', 'bryostatin', 'simvastatin', 'MG-132', 'methotrexate', 'colchicine', 'cytochalasin B', 'AZ258', 'cisplatin'], 'mol_list': None, 'balanced': False, 'trainable_emb': False, 'dataset_name': 'bbbc021', 'n_channels': 3, 'num_workers': 6, 'seed': 42, 'multimodal': False, 'batch_correction': False, 'batch_key': None, 'use_condition_embeddings': False, 'add_controls': False, 'condition_embedding_dimension': None, 'modality_list': ['Compound'], 'image_path': 'project_folder/datasets/bbbc021_unannotated_large', 'data_index_path

In [4]:
config["image_path"] = "../" + config["image_path"]
config["data_index_path"] = "../" + config["data_index_path"]
config["embedding_path"] = "../" + config["embedding_path"]

Create an omega config dict

In [5]:
args = OmegaConf.create(config)

#### Initialize data loader

In [6]:
dataloader = CellDataLoader(args)

['transform_cells_bbbc021_six_drugs.ipynb', 'transform_cells_cpg0000_batch_corr.ipynb', 'transform_cells_rxrx1_batch_corr.ipynb', '__pycache__', 'transform_cells_bbbc021_all_unseen_prediction.ipynb', '.ipynb_checkpoints', 'transform_cells_cpg0000_pert_prediction.ipynb', 'tutorial_utils.py']


In [7]:
train_dataloader = dataloader.train_dataloader()
val_dataloader = dataloader.val_dataloader()

#### Initialize model 

In [8]:
checkpoint_dir = "/home/icb/alessandro.palma/environment/IMPA/IMPA/project_folder/experiments/20241126_a6c37871-014a-4288-adb1-33aa8bf2ae0e_bbbc021_all"

In [9]:
solver = IMPAmodule(args, checkpoint_dir, dataloader)
solver._load_checkpoint(1)

Number of parameters in generator: 24318595
Number of parameters in style_encoder: 16164480
Number of parameters in discriminator: 14341784
Number of parameters in mapping_network: 66112
Initializing embedding_matrix...
Initializing generator...
Initializing style_encoder...
Initializing discriminator...
Initializing mapping_network...
IMPAmodule(
  (embedding_matrix): Embedding(88, 1024)
  (generator): DataParallel(
    (module): Generator(
      (from_rgb): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (encode): ModuleList(
        (0): ResBlk(
          (actv): LeakyReLU(negative_slope=0.2)
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (norm2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Fals

RuntimeError: Error(s) in loading state_dict for StyleEncoder:
	size mismatch for conv.7.weight: copying a param with shape torch.Size([512, 512, 3, 3]) from checkpoint, the shape in current model is torch.Size([512, 512, 4, 4]).

In [10]:
solver

IMPAmodule(
  (embedding_matrix): Embedding(88, 1024)
  (generator): DataParallel(
    (module): Generator(
      (from_rgb): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (encode): ModuleList(
        (0): ResBlk(
          (actv): LeakyReLU(negative_slope=0.2)
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (norm2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
          (conv1x1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): ResBlk(
          (actv): LeakyReLU(negative_slope=0.2)
          (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)